# Using a Stewart Platform as a Motion Simulator
#### Python + ESP32 + 3D CAD + Motion Capture Data

This is a WIP progress!


[<img src="./doc/jupyter_nb/Stewart_IRL_1.gif" width="30%" height="30%">](./doc/jupyter_nb/Stewart_IRL_1.gif)
[<img src="./doc/jupyter_nb/Fusion360.png" width="30%" height="30%">](./doc/jupyter_nb/Fusion360.png)
[<img src="./doc/jupyter_nb/Stewart_IRL_2.gif" width="30%" height="30%">](./doc/jupyter_nb/Stewart_IRL_2.gif)


Import the CSV file containing the positions of a body in 6DOF. A 2DOF example of a person running on a treadmill is provided, of which we scale into the form of

Yaw: Radians

Translation in Z axis: Centimeters scaled according to height of motion simulator

In [33]:
import csv
import numpy as np
import matplotlib.pyplot as plt

# file = open('positions_processed.csv')
file = open('./motion_capture/Badminton_r.csv')

csvreader = csv.reader(file)

rows = []
for row in csvreader:
        rows.append(row)
rows
file.close()
movements = np.array(rows).astype('float')
movements[:,0:3] = movements[:,0:3]/180*np.pi
movements[:,3:6] = movements[:,3:6]/100



# yaw_movements = movements[0,:].astype('float') /180*np.pi
# z_movements = movements[1,:].astype('float')/20

plt.subplot(611); plt.plot(movements[:,0]) 
plt.subplot(612); plt.plot(movements[:,1])
plt.subplot(613); plt.plot(movements[:,2])
plt.subplot(614); plt.plot(movements[:,3]) 
plt.subplot(615); plt.plot(movements[:,4])
plt.subplot(616); plt.plot(movements[:,5])

My approach is then to
1. Use the stewart_py package to calculate the inverse kinematic of the desired position according to the movement data
2. Use py_serial to output the data to a ESP32
3. ESP32 receives serial data and issues I2C command to a pca9685
4. pca9685 commands 6 servo motors to rotate accordingly

This is a very cost efficient solution however if I could redo this project I'd use a Raspberry Pi instead and direcly use it to issue I2C commands to a pca9685

In [35]:
import numpy as np
from src.stewart_controller import Stewart_Platform
import matplotlib.pyplot as plt

import time
import serial

# Initialize serial
ser = serial.Serial()
ser.port = 'COM5'
ser.baudrate = 38400
ser.setDTR(False)
ser.setRTS(False)

# ser.close()
if not ser.isOpen():
    ser.open()

# Call object, 
# These parameters are for my 3D models as provided, you can change your parameters according to your own hardware,
# Stewart_Platform(r_B, r_P, lhl, ldl, Psi_B, Psi_P), where
# r_B = Radius of Base (Bottom)
# r_P = Radius of Platform (Top)
# lhl = Servo Horn Length
# ldl = Rod length
# Psi_B = Half of angle between two anchors on the base
# Psi_P = Half of angle between two anchors on the platform
# For details please refer to my tutorial https://github.com/Yeok-c/Stewart_Py

platform = Stewart_Platform(13.2/2, 17.5/2, 5.08, 13, 0.2269, 0.82)
controller_freq = 10
%matplotlib qt
# Initialize Plots
fig, ax = plt.subplots()    

while 1:
    # Loop through various angles
    for ix in range(0, 2400, 10):
        time.sleep(1/controller_freq)
        # tic = time.time()
# [movements[ix,0], movements[ix,1], movements[ix,2]]
        # send servo angles for this time-step. Movement input data is scaled for better clarity, output is scaled into degrees
        servo_angles = platform.calculate_matrix( np.array([m ovements[ix,3], movements[ix,4], movements[ix,5]-2]), np.array([-movements[ix,0], movements[ix,2], movements[ix,1]]) )/np.pi*180
        
        # Send serial data to ESP32 in the form of a string "<A,float,float,float,float,float,float>"
        towrite = np.array2string(servo_angles, precision=1, separator=',').strip('[]')
        towrite = '<A,' + towrite + '>'
        ser.write(towrite.encode())     # write a string
        print(towrite)

        # Hide this if your computer takes too long to plot, affecting the controller speed
        # ax = platform.plot_platform()

        # plt.pause(1/controller_freq)
        # plt.draw()

        # toc = time.time()
        # print(tic-toc) # Measure time to see if plotting affects speed of controller


<A, 13.7,-34.5, 18.5,-30.5, 16.4,-27.1>
<A, 11.4,-41.4, 17.1,-38.3, 12.6,-33.3>
<A, 17.9,-32.8, 19.3,-40.4,  8.9,-38.1>
<A, 17.3,-32.2, 22.4,-36.2,  7.9,-41.2>
<A, 12.7,-37.6, 24.4,-30.3, 10.8,-39.9>
<A, 16. ,-35.1, 30. ,-25.9, 12.5,-42.7>
<A, 25.2,-29.3, 38. ,-22.5, 18.4,-43.5>
<A, 33.4,-22.7, 44.7,-18.7, 22.9,-44.1>
<A, 36.5,-19.5, 46.4,-17.3, 23.7,-44.4>
<A, 40.9,-16. , 44. ,-22.8, 27.4,-38.1>
<A, 53.1, -5.3, 54.5,-19.2, 36.9,-35.8>
<A, 67.4, 18.2, 66.7,-15.8, 38.9,-41.1>
<A, 70.2, 25. , 70.4,-12.3, 38.6,-43.1>
<A, 64.9, 13.8, 67.7, -7.2, 41.1,-38.9>
<A, 49.1, -5.5, 53.3, -9.6, 37.7,-33.2>
<A, 35.2,-17.6, 37.4,-14.9, 33.8,-24.4>
<A, 27.9,-22.8, 24.3,-21.3, 32. ,-15.6>
<A, 28.9,-23.1, 19.4,-26.4, 34.5, -9.5>
<A, 39.4,-18.1, 35.2,-19.2, 41.8,-13.8>
<A, 55.1, -1. , 52.1,-11.8, 44.5,-23.5>
<A, 65.7, 19.4, 63.4, -7.8, 39.8,-35.4>
<A, 74.5, 36.4, 72.4, -3.6, 40.1,-38. >
<A, 68.8, 19.7, 69. ,  3. , 49.7,-26.3>
<A, 58. ,  5.5, 58.3,  1.2, 48.9,-20.9>
<A, 59.7,  7.9, 56.6, -0.9, 48.3,-20. >


IndexError: index 1500 is out of bounds for axis 0 with size 1499

In [21]:
import numpy as np
from src.stewart_controller import Stewart_Platform
import matplotlib.pyplot as plt
platform = Stewart_Platform(13.2/2, 17.5/2, 5.08, 13, 0.2269, 0.82)
pi = np.pi
%matplotlib qt
print(movements[1,:])
# Initialize Plots
# fig, ax = plt.subplots()    
servo_angles = platform.calculate_matrix( np.array(movements[100,0:3]), np.array(movements[100,3:6]) )/np.pi*180
ax = platform.plot_platform()

plt.draw()

[-0.69110231 -0.37873975 -0.01080202 -0.02784346  0.09805191  0.05957147]


In [3]:
import numpy as np
r_B, r_P, gamma_B, gamma_P = 6.2, 5, 0.2269, 0.82

# 0.2269rad is 13 degrees, which is standard gamma_B for stewart platforms

pi = np.pi
## Define the Geometry of the platform

# Psi_B (Polar coordinates)
psi_B = np.array([ 
    -gamma_B, 
    gamma_B,
    2*pi/3 - gamma_B, 
    2*pi/3 + gamma_B, 
    2*pi/3 + 2*pi/3 - gamma_B, 
    2*pi/3 + 2*pi/3 + gamma_B])

# psi_P (Polar coordinates)
# Direction of the points where the rod is attached to the platform.
psi_P = np.array([ 
    pi/3 + 2*pi/3 + 2*pi/3 + gamma_P,
    pi/3 + -gamma_P, 
    pi/3 + gamma_P,
    pi/3 + 2*pi/3 - gamma_P, 
    pi/3 + 2*pi/3 + gamma_P, 
    pi/3 + 2*pi/3 + 2*pi/3 - gamma_P])

# Coordinate of the points where servo arms 
# are attached to the corresponding servo axis.
B = r_B * np.array( [ 
    [ np.cos(psi_B[0]), np.sin(psi_B[0]), 0],
    [ np.cos(psi_B[1]), np.sin(psi_B[1]), 0],
    [ np.cos(psi_B[2]), np.sin(psi_B[2]), 0],
    [ np.cos(psi_B[3]), np.sin(psi_B[3]), 0],
    [ np.cos(psi_B[4]), np.sin(psi_B[4]), 0],
    [ np.cos(psi_B[5]), np.sin(psi_B[5]), 0] ])
B = np.transpose(B)
    
# Coordinates of the points where the rods 
# are attached to the platform.
P = r_P * np.array([ 
    [ np.cos(psi_P[0]),  np.sin(psi_P[0]), 0],
    [ np.cos(psi_P[1]),  np.sin(psi_P[1]), 0],
    [ np.cos(psi_P[2]),  np.sin(psi_P[2]), 0],
    [ np.cos(psi_P[3]),  np.sin(psi_P[3]), 0],
    [ np.cos(psi_P[4]),  np.sin(psi_P[4]), 0],
    [ np.cos(psi_P[5]),  np.sin(psi_P[5]), 0] ])
P = np.transpose(P)

print('6x3 array for Base anchors \n', B)

print('6x3 array for Platform anchors \n', P)

6x3 array for Base anchors 
 [[ 6.04108436  6.04108436 -1.8126619  -4.22842247 -4.22842247 -1.8126619 ]
 [-1.39474002  1.39474002  5.92910253  4.53436252 -4.53436252 -5.92910253]
 [ 0.          0.          0.          0.          0.          0.        ]]
6x3 array for Platform anchors 
 [[ 4.87150733  4.87150733 -1.46040129 -3.41110604 -3.41110604 -1.46040129]
 [-1.12623991  1.12623991  4.78196906  3.65572915 -3.65572915 -4.78196906]
 [ 0.          0.          0.          0.          0.          0.        ]]


In [4]:
import matplotlib.pyplot as plt
import seaborn

def plot_2D_annotate(X, Y):
    X = list(X)
    Y = list(Y)
    ax.plot(X,Y, 'or')  # Plot Points
    # Annotate Points
    for i, (xy) in enumerate(zip(X, Y)):                                 
        ax.annotate('Point ' + str(i) + '\n(%.5s, %.5s)' % xy, xy=xy, textcoords='data')

    X.append(X[0])
    Y.append(Y[0])
    ax.plot(X,Y)    # Plot lines

seaborn.set(style='ticks')
# plt.style.use('dark_background')

fig = plt.figure(figsize=(14, 6), dpi=80)

ax = fig.add_subplot(121)
ax.set_title('Base Anchors')
plot_2D_annotate(B[0,:], B[1,:])
circle_B = plt.Circle((0, 0), r_B, color='g', fill=False)
ax.add_patch(circle_B)

ax = fig.add_subplot(122)
ax.set_title('Platform Anchors')
plot_2D_annotate(P[0,:], P[1,:])
circle_r = plt.Circle((0, 0), r_P, color='g', fill=False)
ax.add_patch(circle_r)

plt.show()